In [ ]:
import numpy as np
import cv2
from image_geometry import PinholeCameraModel
from sensor_msgs.msg import CameraInfo
from duckietown_utils.yaml_wrap import yaml_load_file

In [ ]:
# Prepare visualisation tools
from __future__ import print_function
%matplotlib inline
#%matplotlib notebook
import matplotlib.pyplot as plt

def bgrshow(bgr, size=(10,4)):
    imshow(cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB), size=size)
    
def imshow(img, size=(10,4), cmap='viridis'):
    plt.figure(figsize=size, dpi= 80, facecolor='w', edgecolor='k')
    plt.imshow(img, cmap = cmap)
    plt.show()

In [ ]:
# Load jpegs
import glob
raw = []
for im_path in sorted(glob.glob("frames/*.png")):
    bgr = cv2.imread(im_path,cv2.IMREAD_COLOR)
    raw.append(bgr)
    
# loads stopline coordinates wrt intersection frame
map_features = yaml_load_file('map/stoplines.yaml')
map_pts = map_features.values()[0]
map_sgs = map_features.values()[1]

In [ ]:
# Prepare the AI filter
from AntiInstagram import AntiInstagram
ai = AntiInstagram()

output_scale = 0.5
color_balance_percentage = 0.8

try:
    raw0 = raw[0]
except (NameError, IndexError) as e:    
    print("No image in raw[0] to calibrate AI filter")
else:
    ai.calculate_color_balance_thresholds(raw0, output_scale, color_balance_percentage)

In [ ]:
H = np.array([-3.589767590401803e-05, -0.00023491693147080584, -0.1374201532070648,
               0.0008247535901185687, -5.00225097638814e-06,   -0.2685590146427659, 
              -5.664504414708967e-05, -0.006300114132830898,    1.0])
H = np.reshape(H,(3,3))

In [ ]:
def camera_info_from_yaml(filename):
    calib_data = yaml_load_file(filename)
    cam_info = CameraInfo()
    cam_info.width = calib_data['image_width']
    cam_info.height = calib_data['image_height']
    cam_info.K = calib_data['camera_matrix']['data']
    cam_info.D = calib_data['distortion_coefficients']['data']
    cam_info.R = calib_data['rectification_matrix']['data']
    cam_info.P = calib_data['projection_matrix']['data']
    cam_info.distortion_model = calib_data['distortion_model']
    return cam_info

def make_undistort_maps(ci):
    K = np.array(ci.K).reshape((3,3))
    D = np.array(ci.D).reshape((1,5))
    R = np.array(ci.R).reshape((3,3))
    P = np.array(ci.P).reshape((3,4))
    mapx = np.ndarray(shape=(ci.height, ci.width, 1), dtype='float32')
    mapy = np.ndarray(shape=(ci.height, ci.width, 1), dtype='float32')
    mapx, mapy = cv2.initUndistortRectifyMap(K, D, R, P, 
                     (ci.width, ci.height), cv2.CV_32FC1, mapx, mapy)
    return mapx, mapy


In [ ]:
ci = camera_info_from_yaml('old_intrinsics.yaml')
mapx, mapy = make_undistort_maps(ci)
pcm = PinholeCameraModel()
pcm.fromCameraInfo(ci)

In [ ]:
hsv_red1 = np.array([  0,140,100])
hsv_red2 = np.array([ 15,255,255])
hsv_red3 = np.array([165,140,100])
hsv_red4 = np.array([180,255,255])
cutoff   = 0
# input is full res raw bitmaps
# output is full res rectified, cropped, stop lines

def preprocess(img, **kwargs):
    rectarg = kwargs.get('rectify')
    if rectarg:
        rect = rectify(img)  # rectify
    else:
        rect = img
    crpd = rect[cutoff:]  # crop
    aied = ai.apply_color_balance(crpd, output_scale)  # color balance
    hsv = cv2.cvtColor(aied, cv2.COLOR_BGR2HSV)       
    bw1 = cv2.inRange(hsv, hsv_red1, hsv_red2)
    bw2 = cv2.inRange(hsv, hsv_red3, hsv_red4)
    red = cv2.bitwise_or(bw1, bw2)  # select red       
    return red

In [ ]:
# nice to always have at hand
reds = []
for rawi in raw[:]:
    reds.append(preprocess(rawi, rectify = False))

In [ ]:
# origin: (x,y) axle coordinates (meters) of the near-left corner of the square
# side:   side in meters of square on the ground 
# w:      side in pixels of the output image
def rect2bird(image, origin, side, w):
    # H assumes incoming image resolution is VGA (640x480) use output_scale if not.
    # pixel at bottom center is x: 0.128 y: 0.00344677851347 #depends on cali
    # x axis is longitudinal, y axis is to the left
    # bottom edge aligned
    # don't think backwards
    oh, ow = np.shape(image) #original width, original height
    side = float(side)
    x0 = origin[0]
    y0 = origin[1]+side/2
    res = float(w)/side #pix/m in square
    
    output = np.zeros((w, w), dtype = np.uint8)
    # loop over input image pixels
    for row in range(oh):
        for column in range(ow):
            if image[row, column] == 0: 
                # if pixel is OFF skip it
                continue
            # if pixel is ON find out where it belongs on our output square
            px = (column/output_scale, row/output_scale+cutoff)
            gnd = rect2axle(px)
            # now I have ground plane coordinates wrt axle frame
            # find correpsonding pixel in square ("ground pixel x and y")
            gpx = int(w-(gnd[0] - x0)*res) # column
            gpy = int( -(gnd[1] - y0)*res) # row

            if gpx < w and gpx >= 0 and gpy < w and gpy >= 0:
                output[gpx, gpy] = image[row, column]
    return output

# Rectifies image using pinhole camera model
def rectify(cv_image_raw):
    cv_image_rectified = np.zeros_like(cv_image_raw)
    if np.shape(cv_image_raw)
    mapx, mapy = cv2.initUndistortRectifyMap(K, D, R, P, 
                     (ci.width, ci.height), cv2.CV_32FC1, mapx, mapy)

    return cv2.remap(cv_image_raw, mapx, mapy, cv2.INTER_NEAREST, cv_image_rectified)

# shortcut for a frequent use case of the two above
def bird(img):
    h,w = np.shape(img)
    rect = rectify(img)
    rect[:,(0,w-1)] = 128
    rect[(0,h-1),:] = 128
    bev = rect2bird(rect,[0, 0], 1.00, 480)
    return bev

In [ ]:
#RULE pixel coordinates are tuples, points are arrays

# axle to rectified pixel coordinates
def axle2rect(point):
    ground_point = np.array([point[0], point[1], 1.0])
    image_point = np.linalg.solve(H, ground_point)
    image_point = image_point / image_point[2]

    return (image_point[0], image_point[1])

def rect2axle(rpx):
    rpx = np.array([rpx[0], rpx[1], 1.0])
    gp = np.dot(H, rpx)
    return np.array([gp[0]/gp[2], gp[1]/gp[2], 0.0])

# pts are lists
def cam2dist(pts):
    rvec = tvec = np.array([[0,0,0]], dtype='float')
    K = pcm.intrinsicMatrix()
    D = pcm.distortionCoeffs()
    ptsOut, _ = cv2.projectPoints(np.array(pts, dtype='float'), rvec, tvec, K, D)
    return ptsOut[:,0,:].tolist()

        
# same as above but doesn't apply distortion    
def cam2rect(pts):
    P = pcm.projectionMatrix()
    pr = []
    for pt in pts:
        pt = np.concatenate([np.array(pt), [1]])
        pri = np.dot(P, pt).tolist()[0]
        pr.append(pri)
    return pr

def rect2cam(pr):
    uv = pcm.projectPixelTo3dRay(pr)
    return np.array([uv[0]/uv[2], uv[1]/uv[2], 1.0])

def rect2dist(rpx):
    cpt = rect2cam(rpx)
    return cam2dist(cpt)

def dist2rect(dist):
    #print("dist:",dist,"type:",type(dist))
    return pcm.rectifyPoint(dist)

from numpy import sin, cos, pi
# pose is (x,y,theta) - (0,0,0) is the target exit pose. 
def axle2intersection(g, pose):
    g = np.array([g[0],g[1]])
    t = pose[2]
    d = np.array([pose[0],pose[1]])
    R = [[cos(t), -sin(t)],[sin(t), cos(t)]]
    i = np.dot(R,g)
    return i + d
 
def intersection2axle(pt, pose):
    pt=[pt[0],pt[1]]
    t = pose[2]
    d = np.array([pose[0],pose[1]])
    R = [[cos(t), sin(t)],[-sin(t), cos(t)]]
    i = np.dot(R, pt-d)
    return np.array([i[0],i[1],0.0])   

def intersection2rect(pt, pose):
    gnd = intersection2axle(pt, pose)
    return axle2rect(gnd)

def rect2intersection(px, pose):
    gnd = rect2axle(px)
    return axle2intersection(gnd, pose)

def dist2intersection(dpt,pose):
    #print("pose:",pose,"type:",type(pose))
    #print("dpt:",dpt,"type:",type(dpt))

    rpt = dist2rect(dpt)
    pt = rect2intersection(rpt, pose)
    return pt

def intersection2dist(pt,pose):
    ipt = intersection2rect(pt,pose)
    cpt = rect2cam(ipt)
    dpt = cam2dist([cpt])[0]
    idpt = (int(dpt[0]), int(dpt[1]))
    return idpt

# input: gnd:    (x,y) coordinates of point in intersection coordinates
#        origin: (x,y) origin of the birds eye view in axle coordinates
#        side:   side of the square captured in the birds eye view 
#        w:      width (equal to height) of the birds eye view image
# output: pixel coordinates of the point in the birds-eye-view 
def intersection2bird(gnd, pose, origin, side, w):
    apt = intersection2axle(gnd, pose)
    brd = axle2bird(apt,origin, side, w)
    return brd

# input: gnd:    (x,y) coordinates of point in axle coordinates
#        origin: (x,y) origin of the birds eye view in axle coordinates
#        side:   side of the square captured in the birds eye view 
#        w:      width (equal to height) of the birds eye view image
# output: pixel coordinates of the point in the birds-eye-view 
def axle2bird(gnd, origin, side, w):
    side = float(side)
    x0 = origin[0]
    y0 = origin[1]+side/2
    res = float(w)/side # pix/m in square
    # now I have ground plane coordinates wrt axel frame
    # find correpsonding pixel in square ("ground pixel x and y")
    gpv = int(w-(gnd[0] - x0)*res) #column
    gpu = int(-(gnd[1] - y0)*res) #row
    return (gpu, gpv)

def dist2bird(dist, origin = [0,0], side=1.0, w=480):
    r = dist2rect(dist)
    a = rect2axle(r)
    return axle2bird(a,origin, side, w)

In [ ]:
pose = [-0.09, -.31, pi/2] # before right turn
red = np.copy(reds[0])

In [ ]:
# red is latest, distorted stoplines image
# prior is naively integrated pose
def estimate(red, prior):
    #1. given the expected pose, where do we expect the vertices to be?
    #   - project the stopline points to the distorted image 
    intersection2dist(pt,pose)

In [ ]:
def get_prior(pose):
    hud_pts = dict()
    for key in map_pts.keys():
        hud_pts[key] = intersection2dist(map_pts[key][1], pose)
    return hud_pts

In [ ]:
def overlay_prior_approx(img, pose):
    hud_pts = dict()
    for key in map_pts.keys():
        hud_pts[key] = intersection2dist(map_pts[key][1], pose)
    for pt in hud_pts.values():
        cv2.drawMarker(img, pt, (0,0,200), 
                       markerType=cv2.MARKER_SQUARE,
                       markerSize=40, 
                       thickness =2, 
                       line_type =cv2.LINE_4)
    return img

def overlay_prior_bev_approx(bev, pose, orig, side, w):
    bev_pts = dict()
    for key in map_pts.keys():
        hud_pts[key] = intersection2bird(map_pts[key][1], 
                                         pose,
                                         orig,side,w
                                        )
        
         
    for pt in hud_pts.values():
        cv2.drawMarker(bev, pt, (120,0,100), 
                       markerType=cv2.MARKER_SQUARE,
                       markerSize=20, 
                       thickness =1, 
                       line_type =cv2.LINE_4)
    return bev

def overlay_prior(img, pose):
    hud_pts = dict()
    for key in map_pts.keys():
        hud_pts[key] = intersection2dist(map_pts[key][1], pose)
    for pt in hud_pts.values():
        cv2.drawMarker(img, pt, (100,100,0), 
                       markerType=cv2.MARKER_SQUARE,
                       markerSize=40, 
                       thickness =2, 
                       line_type =cv2.LINE_4)
    return img

def overlay_prior_bev(bev, pose, orig, side, w):
    bev_pts = dict()
    for key in map_pts.keys():
        bev_pts[key] = intersection2bird(map_pts[key][1], 
                                         pose,
                                         orig,side,w)
    for pt in bev_pts.values():
        cv2.drawMarker(bev, pt, (200,0,200), 
                       markerType=cv2.MARKER_SQUARE,
                       markerSize=20, 
                       thickness =1, 
                       line_type =cv2.LINE_4)
    return bev

In [ ]:
def getcorners(img,radius=35):
    kpcoords = detect_corners(img)
    threshold = radius**2
    bins = [] #list of lists of tuples
    for k in kpcoords:
        placed = False
        for b in bins:
            if (k[0]-b[0][0])**2 + (k[1]-b[0][1])**2 < threshold:
                b.append(k)
                placed = True
                break
        if not placed:
            bins.append([k])  
    return bins

# takes rectified images, finds features, then projects the features only
def getcorners_bev(dist_img,radius=40,viz=False):
    kpdist = detect_corners(dist_img,viz=viz)
    kpbev = [dist2bird(kd) for kd in kpdist]
    threshold = radius**2
    bins = [] #list of lists of tuples
    for k in kpbev:
        placed = False
        for b in bins:
            if (k[0]-b[0][0])**2 + (k[1]-b[0][1])**2 < threshold:
                b.append(k)
                placed = True
                break
        if not placed:
            bins.append([k])   
    return bins

# takes kp in pixel coordinates for img
def overlay_kp(img, kpcoords, color = (100,100,100)):
    out = np.copy(img)
    for kp in kpcoords:
        pix  = (int(kp[0]),int(kp[1]))
        cv2.drawMarker(out, pix, color, 
                       markerType=cv2.MARKER_SQUARE, 
                       markerSize=10, thickness=1, line_type=cv2.LINE_4)
    return out

# takes bev image, kp in distorted coordinates 
def overlay_kp_bev(bev, kpcoords, outlist=None, color = (100,100,100)):
    brdpts = []
    for kp in kpcoords:
        brdpts.append(dist2bird(kp, [0, 0], 1.00, 480))
        if outlist is not None:
            outlist.append(brdpts)
    out = overlay_kp(bev,brdpts,color=color)
    return out


In [ ]:
flatten = lambda a:[k for b in a for k in b]

In [ ]:
sigma = [0.01, 0.01, 0.10] #m m rad
def disturb(pose):
        return pose + np.random.randn(3)*sigma

In [ ]:
red = np.copy(reds[0])

kpcoords = detect_corners(red, viz=False, scale=1.0/3)

pose = np.array([-0.09, -.31, pi/2]) # before right turn
naivepose = disturb(pose)
 
print(np.shape(red))
bev = bird(red)
imshow(bev)

#bev = overlay_kp_bev(bev, kpcoords)
bev = overlay_prior_bev(bev, naivepose, [0, 0], 1.00, 480)
print(np.unique(bev))

In [ ]:
fast = cv2.FastFeatureDetector_create(nonmaxSuppression = False,
                                     #type=cv2.FAST_FEATURE_DETECTOR_TYPE_9_16,
                                      
                                      #type=cv2.FAST_FEATURE_DETECTOR_TYPE_5_8,
                                      #type=cv2.FAST_FEATURE_DETECTOR_TYPE_7_12,
                                      threshold = 75
                                     )
def detect_corners(red, **kwargs):
    viz = kwargs.get('viz')
    scale = kwargs.get('scale')
    if scale is None:
        scale = 1.0/1
    h,w = np.shape(red)
    red = cv2.resize(red, 
                     (int(w*scale), int(h*scale)), 
                      interpolation = cv2.INTER_NEAREST)
    red = cv2.GaussianBlur(red, (3,3), 2)
    kps = fast.detect(red, None)
    if viz:
        imshow(cv2.drawKeypoints(red, kps, None, color=(255,0,0)))
    kpcoords = []
    for kp in kps:
        kpcoords.append((kp.pt[0]/scale, kp.pt[1]/scale))
        #print(kp.pt)
    return kpcoords

In [ ]:
for dimg in reds[0:1]:
    ms = []
    radius = 10
    bevbins = getcorners_bev(dimg, radius, viz=True)
    #bevbins = [[dist2bird(k) for k in db] for db in dbins]
    bev = bird(dimg)
    nbins = len(bevbins)
    print(nbins,': ')
    colstep = 255/(nbins+1)
    col = colstep
    for b in bevbins:
        print("n=",len(b),"\tmiddle=",end='')
        m = np.mean(b,axis=0)
        print(m) 
        ms.append(m)
        bev = overlay_kp(bev, b, color = (col,0,0))

        col += colstep
    imshow(bev,size=(10,10),cmap='tab20')

    #bev = overlay_kp(bev, flatten(bevbins))
#dbins = getcorners(dimg)
#for b in dbins:
#    imshow(overlay_kp(dimg, b))

In [1]:
#!/usr/bin/python

# Standard imports
import cv2
import numpy as np;
from matplotlib import pyplot as plt


# Setup SimpleBlobDetector parameters.
params = cv2.SimpleBlobDetector_Params()

# Change thresholds
params.minThreshold = 0
params.maxThreshold = 80
# Filter by Area.
params.filterByArea = True
params.minArea = 25

# Filter by Circularity
params.filterByCircularity = False
#params.minCircularity = 0.1

# Filter by Convexity
params.filterByConvexity = False
#params.minConvexity = 0.87

# Filter by Inertia
params.filterByInertia = False
#params.minInertiaRatio = 0.01

# Create a detector with the parameters
detector = cv2.SimpleBlobDetector_create(params)


# Read image
for r in raw:
    im = np.copy(255-preprocess(r))#raw[0])#
    # Detect blobs.
    keypoints = detector.detect(im)
    #print(keypoints)
    #imshow(im)
    # Draw detected blobs as red circles.
    # cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures
    # the size of the circle corresponds to the size of blob
    im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (255,0,0), 
                                          cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    titles = ['Blobs Detected']
    images = [im_with_keypoints]


    for i in xrange(1):    
        plt.subplot(1,1,i+1), plt.imshow(images[i],'gray')
        plt.title(titles[i])
        plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis    
    plt.show()

NameError: name 'raw' is not defined

In [ ]:

red = np.copy(reds[0])

kpcoords = detect_corners(red, viz = True,)


imshow(kpimg)

kpint = []
for kp in kpcoords:
    kpint = dist2intersection(kp, pose)
rect = rectify(kpimg)
imshow(rect)
rect[:,(0,640-1)] = 128
rect[(0,480-1),:] = 128

bev = rect2bird(rect[:,:,1],[0, 0], 1.00, 480)

gp = []
for kp in kpcoords:
    kpr = dist2rect(kp)
    g = rect2axle(kpr)
    gp.append(axle2bird(g,[0, 0], 1.00, 480))

for i in range(len(gp)):
    cv2.drawMarker(bev, gp[i], (100,0,0), 
                   markerType=cv2.MARKER_STAR, 
               markerSize=10, thickness=1, line_type=cv2.LINE_AA)

imshow(overlay_prior_bev(bev, naivepose, [0, 0], 1.00, 480))

In [ ]:

kpint = []
for kpc in kpcoords:
    kpi = dist2intersection(kpc, pose)
    kpint.append(kpi)

In [ ]:
raise  # OLD CODE
dis = np.copy(reds[0])
for pt in hud_pts.values():
    cv2.drawMarker(dis, pt, (128,0,0), 
                   markerType=cv2.MARKER_STAR,
               markerSize=40, thickness=1, line_type=cv2.LINE_AA)

In [ ]:
raise # OLD CODE


### works on absolute image pixel values
# BEWARE ARTIFACTS if you try to draw outside of the image bounds
def draw_segment(image, pt0, pt1, color):
    pt0 = (int(pt0[0]), int(pt0[1]))
    pt1 = (int(pt1[0]), int(pt1[1]))
    defined_colors = {
        'red': ['rgb', [1, 0, 0]],
        'green': ['rgb', [0, 1, 0]],
        'blue': ['rgb', [0, 0, 1]],
        'yellow': ['rgb', [1, 1, 0]],
        'magenta': ['rgb', [1, 0, 1]],
        'cyan': ['rgb', [0, 1, 1]],
        'white': ['rgb', [1, 1, 1]],
        'black': ['rgb', [0, 0, 0]]}
    _color_type, [r, g, b] = defined_colors[color]
    cv2.line(image, pt0, pt1, (b * 128, g * 128, r * 128), 3)
    return image

# takes RECTIFIED image and robot pose wrt intersection ref frame
# return the rectified image with stop lines superimposed
def overlay(img, pose):
    # use pose to transform the overlay
    for seg in map_sgs:
        pt0 = intersection2rect(map_pts[seg['points'][0]][1],pose)
        pt1 = intersection2rect(map_pts[seg['points'][1]][1],pose)
        #print(pt0, " - ", pt1)
        col = 'white'#seg['color']
        img = draw_segment(img, pt0, pt1, col)
 
    return img
# takes DISTORTED image and robot pose wrt intersection ref frame
# return the rectified image with stop lines superimposed
def dist_overlay(img, pose):
    # use pose to transform the overlay
    for seg in map_sgs:
        pt0 = intersection2dist(map_pts[seg['points'][0]][1],pose)
        pt1 = intersection2dist(map_pts[seg['points'][1]][1],pose)
        #print(pt0, " - ", pt1)
        col = 'white'#seg['color']
        img = draw_segment(img, pt0, pt1, col)
    # print('segments will be straight, corners have been distorted')
    return img

In [ ]:
img = np.copy(raw[0])
rect = rectify(img)
pose = [-0.09, -.31, pi/2] # before right turn
hud = overlay(rect, pose)
bgrshow(hud)
dist_hud = dist_overlay(img, pose)
bgrshow(dist_hud)

In [ ]:
red = reds[0]

ddepth = cv2.CV_16S
grad_x = cv2.Sobel(red, ddepth, 1, 0, ksize=-1, scale=1, borderType=cv2.BORDER_DEFAULT)
grad_y = cv2.Sobel(red, ddepth, 0, 1, ksize=-1, scale=1, borderType=cv2.BORDER_DEFAULT)
grad = np.stack([grad_x,grad_y])
print(np.shape(grad))
#pcd =    cv2.preCornerDetect(red, ksize=11)
#har =    cv2.cornerHarris(red, 10, ksize=11, k=10)

evnv = cv2.cornerEigenValsAndVecs(red, blockSize=1, ksize=1) #(l1,l2,x1,y1,x2,y2)

l1 = evnv[:,:,0]
l2 = evnv[:,:,1]
x1 = evnv[:,:,2]
y1 = evnv[:,:,3]
theta1 = np.arctan2(y1,x1)
x2 = evnv[:,:,4]
y2 = evnv[:,:,5]
theta2 = np.arctan2(y2,x2)

theta_grad = np.arctan2(grad[1],grad[0])

#imshow(theta_grad)
#imshow(grad_x)
print(grad_x[300,300])


In [ ]:
def dispOpticalFlow( image,Flow ):
    
    h, w = np.shape(image)
    mask = np.zeros_like(image)
    
    for i in range(h):
        for j in range(w):
            if Flow[0,i,j] != 0 or Flow[1,i,j] != 0:
                if (i+j)%5 != 0: continue
                j_ = int(j + 0.003*Flow[0,i,j])
                i_ = int(i + 0.003*Flow[1,i,j])
                i_ = np.clip(i_, 0, h)
                j_ = np.clip(j_, 0, w)
                mask = cv2.line(mask, (j,i),(j_,i_), [255, 255, 255], 1)
    #superpose lines onto image
    #img = cv2.add(image,mask)
    #print image
    imshow(mask)
    return 


In [ ]:

cv2.__version__


In [ ]:
deltas = []
for i in range(len(reds)-1):
    deltas.append(reds[i] + 0.5*reds[i+1])
    
for delta in deltas:
    imshow(delta)

In [ ]:
intp1 = [-0.21,0.12,0]
intp2 = [0,0,0]
pose = [-0.09, -.31, pi/2] # before right turn
middle = intersection2axle(intp1,pose)
target = intersection2axle(intp2,pose)

# axle coordinates, xyz in meters
# x forward, y left, z up
gnd = [(0.128+0.04, 0.003+0.12, 0), middle, target]
pr0 = []
for g in gnd:
    pr0.append(axle2rect(g)) # NOTE: you are rounding to integers but don't need to
print('Rectified pixle coordinates = ', pr0)

pts = []
for pr0i in pr0:
    pts.append(rect2cam(pr0i))
print('Camera frame coordinates = ', pts)

pd = cam2dist(pts)
print('Distorted pixel coords = \n', pd)

pr = cam2rect(pts)
print('Rectified pixel coord = \n', pr)

dis = np.copy(raw[0])
for i in range(len(pd)):
    cv2.drawMarker(dis, (int(pd[i][0]),int(pd[i][1])), (255,0,0), 
                   markerType=cv2.MARKER_STAR,
               markerSize=40, thickness=3, line_type=cv2.LINE_AA)
 
rec = rectify(dis)

bgrshow(dis)

for i in range(len(pr)):
    cv2.drawMarker(rec, (int(pr[i][0]),int(pr[i][1])), (0,255,0), 
                   markerType=cv2.MARKER_STAR, 
               markerSize=20, thickness=1, line_type=cv2.LINE_AA)
for i in range(len(pr)):
    cv2.drawMarker(rec, (int(pr0[i][0]),int(pr0[i][1])), (255,255,0), 
                   markerType=cv2.MARKER_STAR, 
               markerSize=20, thickness=2, line_type=cv2.LINE_AA)

bgrshow(rec)

red = preprocess(dis)
bev = rect2bird(red,[0, 0], 1.00, 480)

gp = []
for g in gnd:
    gp.append(axle2bird(g,[0, 0], 1.00, 480))

for i in range(len(gp)):
    cv2.drawMarker(bev, (int(gp[i][0]),int(gp[i][1])), (255,255,255), 
                   markerType=cv2.MARKER_STAR, 
               markerSize=10, thickness=1, line_type=cv2.LINE_AA)

imshow(bev)

In [ ]:
red = cv2.resize(reds[0], (320,240))

ddepth = cv2.CV_16S
grad_x = cv2.Sobel(red, ddepth, 1, 0, ksize=-1, scale=1, borderType=cv2.BORDER_DEFAULT)
grad_y = cv2.Sobel(red, ddepth, 0, 1, ksize=-1, scale=1, borderType=cv2.BORDER_DEFAULT)
grad = np.stack([grad_x,grad_y])

dispOpticalFlow(red, grad)

In [ ]:
intp1 = [-0.21,0.12,0]
intp2 = [0,0,0]
pose = [-0.09, -.31, pi/2] # before right turn
middle = intersection2axle(intp1,pose)
target = intersection2axle(intp2,pose)

# axle coordinates, xyz in meters
# x forward, y left, z up
gnd = [(0.128+0.04, 0.003+0.12, 0), middle, target]
pr0 = []
for g in gnd:
    pr0.append(axle2rect(g)) # NOTE: you are rounding to integers but don't need to
print('Rectified pixle coordinates = ', pr0)

pts = []
for pr0i in pr0:
    pts.append(rect2cam(pr0i))
print('Camera frame coordinates = ', pts)

pd = cam2dist(pts)
print('Distorted pixel coords = \n', pd)

pr = cam2rect(pts)
print('Rectified pixel coord = \n', pr)

dis = np.copy(raw[0])
for i in range(len(pd)):
    cv2.drawMarker(dis, (int(pd[i][0]),int(pd[i][1])), (255,0,0), 
                   markerType=cv2.MARKER_STAR,
               markerSize=40, thickness=3, line_type=cv2.LINE_AA)

rec = rectify(dis)

bgrshow(dis)

for i in range(len(pr)):
    cv2.drawMarker(rec, (int(pr[i][0]),int(pr[i][1])), (0,255,0), 
                   markerType=cv2.MARKER_STAR, 
               markerSize=20, thickness=1, line_type=cv2.LINE_AA)
for i in range(len(pr)):
    cv2.drawMarker(rec, (int(pr0[i][0]),int(pr0[i][1])), (255,255,0), 
                   markerType=cv2.MARKER_STAR, 
               markerSize=20, thickness=2, line_type=cv2.LINE_AA)

bgrshow(rec)

red = preprocess(dis)
bev = rect2bird(red,[0, 0], 1.00, 480)

gp = []
for g in gnd:
    gp.append(axle2bird(g,[0, 0], 1.00, 480))

imshow(bev)    
    
for i in range(len(gp)):
    cv2.drawMarker(bev, (int(gp[i][0]),int(gp[i][1])), (255,255,255), 
                   markerType=cv2.MARKER_STAR, 
               markerSize=10, thickness=1, line_type=cv2.LINE_AA)

imshow(bev)

In [ ]:
red = np.copy(reds[0])
  
kpcoords = detect_corners(red, viz=False, scale=1.0/3)

pose = np.array([-0.09, -.31, pi/2]) # before right turn
naivepose = disturb(pose)

#kpimg = overlay_kp(red, kpcoords)
#kpimg = overlay_prior(kpimg, naivepose)
#imshow(kpimg)

kpint = []
for kpc in kpcoords:
    kpi = dist2intersection(kpc, pose)
    kpint.append(kpi)
rect = rectify(red)
rect[:,(0,640-1)] = 128
rect[(0,480-1),:] = 128
bev = rect2bird(rect[:,:],[0, 0], 1.00, 480)
bev = overlay_kp_bev(bev, kpint)
bev = overlay_prior_bev(bev, naivepose, [0, 0], 1.00, 480)
imshow(bev)